#### Project 6

In this project you will simulate secure communication between two parties. There are two roles:  A *sender* and a *receiver*.  The sender wants to do the following things:

1. Digitally sign a message (file) using [RSA PSS](https://en.wikipedia.org/wiki/Probabilistic_signature_scheme) to produce a file called `msg.sig`.
1. Encrypt the message using AES with a 128 bit key $K$ to produce a file called `msg.crypt`.
1. Encrypt $K$ using RSA with the receiver's public key and save to a file called `symkey.crypt`.
1. "Transmit" `msg.sig`,`msg.crypt`,`sym.key` (you don't actually have to do anything for this step).

The receiver wants to do the following:

1. Use her private key to decrypt `symkey.crypt` to produce `symkey`.
1. Use `symkey` to decrypt `msg.crypt` to produce `msg`.
1. Use the sender's public key to verify that `msg.sig` is a valid signature for `msg`.

You should produce two applications that work from the command line: **send** and **receive**.

The **send** program should take three command line inputs, all of which are filenames:
1. The name of the message file.
2. The name of the sender's private key file (for signing).
3. The name of the receiver's public key file (for encrypting the symmetric key).

There is code below that you can adapt for producing RSA keys (public and private) as well as a cryptographically secure 128 bit AES key.

Please name you files as follows.  **Please actually use these names:)  (many people don't but you *must*).**

1. Call the AES key file `symkey`.
1. Call the sender's private key `send_priv.der`  (we will use keys in the DER format).
1. Call the sender's public key `send_pub.der`.
1. Call the receiver's private key `rec_priv.der`.
1. Call the receiver's public key `rec_pub.der`.

The **receive** program should take three command line inputs, all of which are file

In [2]:
### Warning:  Kernel must be Python 3 (System Wide)
### Select this from Kernel pulldown menu in Cocalc


In [1]:
!openssl rand 16 > symkey
!openssl rand 16 > IV


In [3]:
!openssl genrsa -out private.pem 2048
!openssl rsa -in private.pem -out private.der -outform DER
!openssl rsa -in private.pem -outform DER -pubout -out public.der
!openssl rand 16 > aes.key

Generating RSA private key, 2048 bit long modulus (2 primes)
.............................

.............

...................................

........+++++
....

..................+++++
e is 65537 (0x010001)


writing RSA key


writing RSA key


In [3]:

### Adapted from
### https://support.ssl.com/Knowledgebase/Article/View/19/0/der-vs-crt-vs-cer-vs-pem-certificates-and-how-to-convert-them
### and
### https://pycryptodome.readthedocs.io/en/latest/src/signature/pkcs1_pss.html



In [4]:

from Crypto.Signature import pss
from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA
from Crypto import Random

message = 'big marshmallow is alive with flavor'


def sign_message(message,privkeyfile_der):
    key = RSA.import_key(open(privkeyfile_der,'rb').read())
    h = SHA256.new(message.encode())
    signature = pss.new(key).sign(h)
    return signature
    
sig = sign_message(message,"private.der")    
sig

b"\x91\xfc\xea[\x87\xf2+I\x95\x9eDo\xa6\xe0l\xecS\x1e\xb7\xc2-\xd0\xce~\x96\xb9s\xe4\x9a\xe2$\x7f\xbe#;\x9e\xeb6x\x1dc\xf3n\xf7\xa6)\xcf\x86\x86\xf7\xa5\xda=c\xb6\xcaj\x9f[t\x85\xe5\x85\x86\x85f\xaeP\xa4y \x91I\x80\x82>o\xe9\x13'\x0e\x1aG=\xf81\xfd\xe2\xa6\x1d\x83)\x06\xe2\xeb5m\x9e\xac<z\x99\xb8\xc8\xc0\x175\xda\xee\xaf*\x16z\xb8\x01\x87\xa6\xcc\xa6\xdd B}Yc\xc3\xf4\xd7\x85\x86}E\xa1\xd0\x9d\x890\xa6~w9\x05\xe5D\xb6<\xa1\x98Vo\x9f\x0e\x9b\x9c\xfcxCn\x99p\x96\xba\xe4\x81[\xaaZ\x15<W\x0c\xca\xc3\xb6\x93H\xa3@\xd1\xe6\x1f ~Sd\xad\xa83F\xda\xc3\x7f\x87\x9f\xeeE\xa0\xd8\x1f\xa7\x95u\x14\x84\x94\xf7l\xb5@N\xa9\x85\xa2\xd0fpF\xc6i\xdb1ul\x87)\xe9\x05\xec#T[\xd4\x98\x854K^\xe9Fi\x7f\xc0\xc5\x93\x94:\x1c\\\xa4\x88(\x9b\xa4v\x0e\xb2"

In [5]:


def verify_sig(message,signature,pubkeyfile_der):
    key = RSA.import_key(open(pubkeyfile_der,'rb').read())
    h = SHA256.new(message.encode())
    verifier = pss.new(key)
    try:
        verifier.verify(h, signature)
        print( "The signature is authentic.")
    except (ValueError, TypeError):
        print( "The signature is not authentic.")

verify_sig(message,sig,"public.der")

The signature is authentic.


In [6]:
from Crypto.Cipher import PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Hash import SHA

message = open("aes.key",'rb').read()
message = b"For testing purposes, remove this line"

def RSAencrypt(message,pubkeyfile_der):
    key = RSA.importKey(open(pubkeyfile_der,'rb').read())
    cipher = PKCS1_OAEP.new(key)  ### Padding Scheme
    ciphertext = cipher.encrypt(message)
    return ciphertext
ciphertext = RSAencrypt(message,"public.der")
ciphertext

b'fu~jI\xd0\xda$\xf2i\xcc\xd5Q\xad\x88\x07{5\xde\xce\x8e\x8b\xf8\xc4\xb1}P=\xda\xd4\xe5\x0e\xed\xc9\xfa\x81\xdbL\'o\x04\x7f\xe6<1\xeb\xc6\x9a\x03\xaf1\x87\x99|-\x05\xa8\xe4\x94f/A\xc4\xe5]\xd9\xf1\xb8\x01\x1c^\xa7\x8b\xae\xf8\x85\x84\x80\x04\xaa\xf3\x9b\xd1\x89\xbe\xa9\xe5\x08m\t\xba\x1c<\xa8j\xc1{\xaf}\x84F\xc8]$\x01\x83\xa2\xe9@\xf6\xe6\xa7\xd5\x9a\x0c\xd5\xf8\xfe\xbd\x0el1\xe1\x84\xa2*A\xbfr\x94\xd1/\x00\\\x15\x9bx\xd2\xbd\xda\x86jp\x86\x17\x00L\xbbg\x9a\xe5S\xec\x8f\x05[\xd3\xcd\xd8\xe0"+\x14\x97\xa7iF\xfa\x16\x05o\x01=Y\xd9a\x17>\xef\xb3\x8e\x983\xb0\xe7\xf2@\xc2mc@#5j\xeeY\xdd\xa5\x8d\xac\x02\x8at\xd3g\x986\x93j\xad)\x95\xeb\xd0\x0fa_\xd1\xf5W$f\xbf\xf1\x0c\xa6@\xd4\x97\xca\x88\x07\xfb\xf5\xc7\x8c\xff\x1a\xbc\xf2\xb1&\x9er\xe3\x9b\xe0\x04N\x87Sd\x88\xd0\x14{'

In [7]:
from Crypto.Hash import SHA
from Crypto import Random

def RSAdecrypt(ciphertext,privatekey_der):
    key = RSA.importKey(open(privatekey_der,'rb').read())
    #dsize = SHA.digest_size
    #sentinel = Random.new().read(15+dsize)      # Let's assume that average data length is 15
    cipher = PKCS1_OAEP.new(key)
    message = cipher.decrypt(ciphertext)
    return message

RSAdecrypt(ciphertext,"private.der")

b'For testing purposes, remove this line'